In [1]:
import json

import polars as pl

from spells import summon, card_df
from spells.enums import ColName, ColType
from spells.columns import ColumnSpec

### Metagame Summary

In this notebook I'll demonstrate the metagame summaries I like to look at, along the lines of the "Deck Color" page on 17Lands.com. First, lets generate that table. I like to only look at the top player cohort to judge what is effective for a winning strategy. Here are recipes for the basic charts

In [2]:

filter_spec = {ColName.PLAYER_COHORT: 'Top'}
columns = [ColName.NUM_WON, ColName.NUM_GAMES, ColName.GAME_WR]

print(summon("DSK", columns=columns, group_by=[ColName.NUM_COLORS], filter_spec=filter_spec))

two_color_filter = {'$and': [{'num_colors': 2}, filter_spec]}
print(summon("DSK", columns=columns, group_by=[ColName.MAIN_COLORS], filter_spec=two_color_filter))

shape: (5, 4)
┌────────────┬─────────┬───────────┬──────────┐
│ num_colors ┆ num_won ┆ num_games ┆ game_wr  │
│ ---        ┆ ---     ┆ ---       ┆ ---      │
│ u32        ┆ u32     ┆ u32       ┆ f64      │
╞════════════╪═════════╪═══════════╪══════════╡
│ 1          ┆ 1803    ┆ 2873      ┆ 0.627567 │
│ 2          ┆ 112978  ┆ 182958    ┆ 0.617508 │
│ 3          ┆ 12108   ┆ 20721     ┆ 0.584335 │
│ 4          ┆ 337     ┆ 600       ┆ 0.561667 │
│ 5          ┆ 17      ┆ 37        ┆ 0.459459 │
└────────────┴─────────┴───────────┴──────────┘
shape: (10, 4)
┌─────────────┬─────────┬───────────┬──────────┐
│ main_colors ┆ num_won ┆ num_games ┆ game_wr  │
│ ---         ┆ ---     ┆ ---       ┆ ---      │
│ str         ┆ u32     ┆ u32       ┆ f64      │
╞═════════════╪═════════╪═══════════╪══════════╡
│ BG          ┆ 9587    ┆ 15624     ┆ 0.613607 │
│ BR          ┆ 11486   ┆ 18595     ┆ 0.617693 │
│ RG          ┆ 16179   ┆ 26464     ┆ 0.611359 │
│ UB          ┆ 4104    ┆ 6945      ┆ 0.590929 │
│ 

You'll note the winrates are higher than the "Top" cohort on 17Lands. That's because we are compelled to use an in-sample cohort. We might want some additional context about the archetypes, like how often the decks splash, the average number of lands, and the average converted mana cost. Let's add all three. There isn't a `GAME_SUM` column built in for splash count, so we'll define one. Note that `GAME_SUM` columns can only be used when no card attributes are used. So we'll need a separate aggregation.

In [3]:
ext = [
    ColumnSpec(
        name='num_splash',
        col_type=ColType.GAME_SUM,
        expr=pl.col(ColName.HAS_SPLASH),
        dependencies=[ColName.HAS_SPLASH]
    ),
    ColumnSpec(
        name='splash_rate',
        col_type=ColType.AGG,
        expr=pl.col('num_splash') / pl.col(ColName.NUM_GAMES),
        dependencies=['num_splash', ColName.NUM_GAMES],
    ),
]
print(by_num_colors_df := summon("DSK", columns=columns + ['splash_rate'], group_by=[ColName.NUM_COLORS], filter_spec=filter_spec, extensions=ext))
print(by_color_pair_df := summon("DSK", columns=columns+['splash_rate'], group_by=["main_colors"], filter_spec=two_color_filter, extensions=ext))
        

shape: (5, 5)
┌────────────┬─────────┬───────────┬──────────┬─────────────┐
│ num_colors ┆ num_won ┆ num_games ┆ game_wr  ┆ splash_rate │
│ ---        ┆ ---     ┆ ---       ┆ ---      ┆ ---         │
│ u32        ┆ u32     ┆ u32       ┆ f64      ┆ f64         │
╞════════════╪═════════╪═══════════╪══════════╪═════════════╡
│ 1          ┆ 1803    ┆ 2873      ┆ 0.627567 ┆ 0.788723    │
│ 2          ┆ 112978  ┆ 182958    ┆ 0.617508 ┆ 0.306595    │
│ 3          ┆ 12108   ┆ 20721     ┆ 0.584335 ┆ 0.15757     │
│ 4          ┆ 337     ┆ 600       ┆ 0.561667 ┆ 0.526667    │
│ 5          ┆ 17      ┆ 37        ┆ 0.459459 ┆ 0.0         │
└────────────┴─────────┴───────────┴──────────┴─────────────┘
shape: (10, 5)
┌─────────────┬─────────┬───────────┬──────────┬─────────────┐
│ main_colors ┆ num_won ┆ num_games ┆ game_wr  ┆ splash_rate │
│ ---         ┆ ---     ┆ ---       ┆ ---      ┆ ---         │
│ str         ┆ u32     ┆ u32       ┆ f64      ┆ f64         │
╞═════════════╪═════════╪═══════════╪

And the mana cost and land count columns are built in, but we have to leave out the `GAME_SUM` columns. We can join the polars dfs if we like.

In [4]:
card_sum_df_numc = summon("DSK", columns=[ColName.DECK_LANDS_AVG, ColName.DECK_MANA_VALUE_AVG], group_by=[ColName.NUM_COLORS], filter_spec=filter_spec)

AssertionError: Invalid manifest for GAME_SUM column num_games

In [9]:
summon("DSK", group_by=["user_game_win_rate_bucket"], columns=["num_games"], filter_spec={'user_n_games_bucket': 500})

user_game_win_rate_bucket,num_games
f64,u32
0.36,611
0.38,550
0.42,2189
0.44,825
0.46,3630
…,…
0.6,5199
0.62,4653
0.64,1019
